## 3. 특정 영역 자르기 (Subsetting)

### 3.1 왜 영역을 잘라야 하는가?
- **메모리 절약**
- **계산 속도**: 필요한 영역만 처리하면 분석이 훨씬 빠르다.

### 3.2 쿠로시오 박스 정의
쿠로시오 정의(임의) 영역: **115°E-160°E, 20°N-50°N**

In [1]:
# 기본 import template
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 (필요시)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 100

In [2]:
# data load
ds_sst = xr.open_dataset('../data/raw/oisst/oisst_monthly_2015_2024_kuroshio.nc')
ds_mld = xr.open_dataset('../data/raw/glorys/glorys_mld_2015_2024_kuroshio.nc')
ds_thetao = xr.open_dataset('../data/raw/argo/glorys_temp_3d_2015_2024_kuroshio_0-1000m.nc')

In [3]:
# 영역 좌표 정의
LON_MIN, LON_MAX = 115, 160
LAT_MIN, LAT_MAX = 20, 50

In [6]:
def subset_kuroshio(ds, lon_var='lon', lat_var='lat'):
    """
    Parameters:
    ds : xarray.Dataset / DataArray
    lon_var, lat_var : str
    
    Returns:
    xarray.Dataset or DataArray
    """
    return ds.sel(
        {lon_var : slice(LON_MIN, LON_MAX),
         lat_var : slice(LAT_MIN, LAT_MAX)}
    )

### 3.3 좌표/차원 이름 /범위 통일
- 경도 체계가 0~360인지 -180~+180인지 알아야 함.

In [4]:
# 변수 확인 data."data_vars"
print(list(ds_sst.data_vars))
print(list(ds_mld.data_vars))
print(list(ds_thetao.data_vars))

['sst']
['mlotst']
['thetao']


In [5]:
# coords
print(list(ds_sst.coords))
print(list(ds_mld.coords))
print(list(ds_thetao.coords))

['lon', 'lat', 'time']
['longitude', 'latitude', 'time']
['longitude', 'latitude', 'time', 'depth']


In [ ]:
print(ds_sst['sst'].dims)
print(ds_mld['mlotst'].dims)
print(ds_thetao['thetao'].dims)

('time', 'lat', 'lon')
('time', 'latitude', 'longitude')
('time', 'depth', 'latitude', 'longitude')


In [7]:
# ds_mld, ds_thetao 이름 쉽게 변경해주기.
ds_mld = ds_mld.rename({
    'longitude' : 'lon',
    'latitude' : 'lat'
})
print(ds_mld.coords)

ds_thetao = ds_thetao.rename({
    'longitude' : 'lon',
    'latitude' : 'lat',
    'depth' : 'z',
})

Coordinates:
  * lon      (lon) float32 2kB 115.0 115.1 115.2 115.2 ... 159.8 159.9 160.0
  * lat      (lat) float32 1kB 20.0 20.08 20.17 20.25 ... 49.75 49.83 49.92 50.0
  * time     (time) datetime64[ns] 960B 2015-01-01 2015-02-01 ... 2024-12-01


In [8]:
# 경도 범위가 확인.
print("lon range: ", ds_sst.lon.values.min(), "~", ds_sst.lon.values.max())
print("lon range: ", ds_mld.lon.values.min(), "~", ds_mld.lon.values.max())
print("lon range: ", ds_thetao.lon.values.min(), "~", ds_thetao.lon.values.max())

lon range:  115.125 ~ 159.875
lon range:  115.0 ~ 160.0
lon range:  115.0 ~ 160.0


In [12]:
# 만약 경도 변환이 필요하다면?
# assign_coords → 값만 바꾸고 데이터는 유지
# ds.sortby(~) → 값이 섞일테니까
'''
def convert_lon360_to_lon180(ds):
    ds = ds.assign_coords(lon=(((ds.lon+180)%360)-180))
    return ds.sortby('lon')
    
def convert_lon180_to_lon360(ds):
    ds = ds.assign_coords(lon=(ds.lon%360))
    return ds.sortby('lon')
'''

"\ndef convert_lon360_to_lon180(ds):\n    ds = ds.assign_coords(lon=(((ds.lon+180)%360)-180))\n    return ds.sortby('lon')\n    \ndef convert_lon180_to_lon360(ds):\n    ds = ds.assign_coords(lon=(ds.lon%360))\n    return ds.sortby('lon')\n"

### 3.4 위도 방향 주의하기

In [9]:
# ds_sst
print(ds_sst.lat.values[:5], "...", ds_sst.lat.values[-5:])
# ds_mld
print(ds_mld.lat.values[:5], "...", ds_mld.lat.values[-5:])
# ds_thetao
print(ds_thetao.lat.values[:5], "..,", ds_thetao.lat.values[-5:])

[20.125 20.375 20.625 20.875 21.125] ... [48.875 49.125 49.375 49.625 49.875]
[20.       20.083334 20.166666 20.25     20.333334] ... [49.666668 49.75     49.833332 49.916668 50.      ]
[20.       20.083334 20.166666 20.25     20.333334] .., [49.666668 49.75     49.833332 49.916668 50.      ]


In [10]:
# 기왕 하는 김에 경도도 체크해봤다.
print(ds_sst.lon.values[:5], "...", ds_sst.lon.values[-5:])
print(ds_mld.lon.values[:5], "...", ds_mld.lon.values[-5:])
print(ds_thetao.lon.values[:5], "..,", ds_thetao.lon.values[-5:])

[115.125 115.375 115.625 115.875 116.125] ... [158.875 159.125 159.375 159.625 159.875]
[115.       115.083336 115.166664 115.25     115.333336] ... [159.66667 159.75    159.83333 159.91667 160.     ]
[115.       115.083336 115.166664 115.25     115.333336] .., [159.66667 159.75    159.83333 159.91667 160.     ]


In [ ]:
# 만약 위도 뒤집기를 해야한다면?

''' 
def safe_subset(ds, lon_range, lat_range, lon_var = 'lon', lat_var ='lat'):
    lon_min, lon_max = lon_range
    lat_min, lat_max = lat_range
    
    # 위도가 감소하는 순서인지 확인해보자.
    if ds[lat_var].values[0] > ds[lat_var].values[-1]:
        lat_slice = slice(lat_max, lat_min) # 뒤집어
    else:
        lat_slice = slice(lat_min, lat_max)
        
    return ds.sel({lon_var : slice(lon_min, lon_max), lat_var: lat_slice})
'''

### 3.5 격자 일치 여부 확인해보기 - 전처리

In [11]:
print(ds_sst.lon.size, ds_mld.lon.size, ds_thetao.lon.size)

180 541 541


In [12]:
print(ds_sst.lat.size, ds_mld.lat.size, ds_thetao.lat.size)

120 361 361


In [15]:
print(float(ds_sst.lon.diff('lon').median()), 
float(ds_mld.lon.diff('lon').median()), 
float(ds_thetao.lon.diff('lon').median()))

0.25 0.0833282470703125 0.0833282470703125


In [16]:
float(ds_sst.lat.diff('lat').median()), float(ds_mld.lat.diff('lat').median())

(0.25, 0.08333206176757812)

---
---
---

# 3 regrid (ipynb 통합)

In [ ]:
# 경로
from pathlib import Path
outdir = Path("../data/processed")
outdir.mkdir(parents=True, exist_ok=True)

In [ ]:
print(type(ds_sst.time.min().item()), ds_sst.time.dtype)
print(type(ds_mld.time.min().item()), ds_mld.time.dtype)
print(t0, type(t0))

In [ ]:
t0 = np.datetime64(max(ds_sst.time.min().values, ds_mld.time.min().values)) 
t1 = np.datetime64(min(ds_sst.time.max().values, ds_mld.time.max().values))

ds_sst2 = ds_sst.sel(time = slice(t0, t1)).sortby('time')   
ds_mld2 = ds_mld.sel(time = slice(t0, t1)).sortby('time')

In [ ]:
print(ds_sst.time.min().values, type(ds_sst.time.min().values))
print(ds_sst.time.to_index().__class__)

### A. Interp: GLORYS MLD -> SST 격자
- 빠르게 분석하는 용도

### B. xESMF regriding : 격자 평균에 가까운 방식
- 고해상도 정보를 저해상도 격자 셀로 면적 평균에 가깝게 내리기
- MLD같은 경우 `conservative_normed` 활용